In [40]:
from pyod.models.knn import KNN
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import os
from scipy import stats

all_files = glob.glob(os.path.join(".", "*.csv"))

df_from_each_file = (pd.read_csv(f, sep=",") for f in all_files)
df_elec = pd.concat(df_from_each_file, axis=0, ignore_index=True)

In [103]:
df_elec['Month'] = df_elec['Fecha'].str.slice(5, 7).str.replace("-", "").astype(int)
X = df_elec[['Month', 'Active_energy']]

scaler = MinMaxScaler()
X['Active_energy'] = scaler.fit_transform(X[['Active_energy']])

<ipython-input-103-45b3cd0764dc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Active_energy'] = scaler.fit_transform(X[['Active_energy']])


In [57]:
outliers_fraction = 0.02

# Cree el modelo con los parámetros definidos y entrénelo
model = KNN(method='mean', contamination=outliers_fraction)
model.fit(X)

scores_pred = model.decision_function(X)
y_pred = model.predict(X)
n_inliers = len(y_pred) - np.count_nonzero(y_pred)
n_outliers = np.count_nonzero(y_pred == 1)

In [104]:
def predict_perdida_no_tecnica(config, scaler, model):
    if type(config) == dict:
        df = pd.DataFrame(config)
    else:
        df = config

    df['Active_energy'] = scaler.transform(df[['Active_energy']])

    print(df)

    y_pred = model.predict(df)

    if y_pred == 0:
        return 'Comportamiento normal'
    elif y_pred == 1:
        return 'Posible perdida no tecnica'

In [108]:
config = {
    'Month': [12], # 1- January, 2- February
    'Active_energy':  [8]
}
predict_perdida_no_tecnica(config, scaler, model)

   Month  Active_energy
0     12       0.547096


'Comportamiento normal'

In [109]:
import pickle

# Save to file in the current working directory
pkl_filename = "pnt.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

# Calculate the accuracy score and predict target values
Ypredict = pickle_model.predict(X)

In [110]:
##loading the model from the saved file
pkl_filename = "pnt.pkl"
with open(pkl_filename, 'rb') as f_in:
    model = pickle.load(f_in)

predictValue = predict_perdida_no_tecnica(config, scaler, model)
predictValue

   Month  Active_energy
0     12       0.547096


'Comportamiento normal'